In [1]:
###############################################################
# LangChain: Agents
# 
# Outline:
# 
# * Using built in LangChain tools: DuckDuckGo search and 
#   Wikipedia
# * Defining your own tools
###############################################################

In [3]:
import os
from openai import OpenAI
# read the api key from environment variable
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be 
# set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


In [6]:
##############################################################
# Built-in LangChain tools
###############################################################

In [7]:
!pip install -U wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=230e3f192690a6cec6ce6483e07f84e99641eea0e0ecbd20444497d9d6f82283
  Stored in directory: /Users/rumnraisin/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [12]:
!pip install -U langchain_experimental

  Obtaining dependency information for langchain_experimental from https://files.pythonhosted.org/packages/26/40/c4571740547ecff9776921c0f7bcecf18337d69ca01b0b625a65539207b1/langchain_experimental-0.0.53-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.7/173.7 kB 3.6 MB/s eta 0:00:00a 0:00:01


In [22]:
!pip install -U langchain-openai

  Obtaining dependency information for langchain-openai from https://files.pythonhosted.org/packages/81/63/012be16114559243aabcc9ec570366df84591dc9f8f3c2349a398e9b3626/langchain_openai-0.0.8-py3-none-any.whl.metadata


In [20]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.tools.python.tool import PythonREPL
from langchain.chat_models import ChatOpenAI

In [23]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [24]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [27]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)


In [28]:
agent("What is the 25% of 300?")

/Users/rumnraisin/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I need to use a calculator to find the answer to this math problem.

Action:
```
{
  "action": "Calculator",
  "action_input": "300*0.25"
}
```


Observation: Answer: 75.0
Thought:I have successfully calculated the answer to the math problem.

Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [29]:
##############################################################
# Wikipedia example
# - Showcase of the agent's interaction with Wikipedia.
# - Example involves asking the agent about Tom 
#   M. Mitchell, an American computer scientist.
# - Demonstration of the agent's ability to leverage 
#   tools to retrieve accurate and contextually 
#   appropriate answers.
###############################################################

In [30]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie \
Mellon University (CMU)\
what book did he write?"

result = agent(question) 




> Entering new AgentExecutor chain...
Thought: I should use the wikipedia tool to search for Tom M. Mitchell's works.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed a

In [31]:
##############################################################
# Python Agent
###############################################################

In [32]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)


In [33]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [35]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 




> Entering new AgentExecutor chain...
I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python_REPL
Action Input: 
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted(customers, key=lambda x: (x[1], x[0]))
```
Observation: 
Thought:The output is a sorted list of customers by last name and then first name.
Action: Python_REPL
Action Input: `print(sorted(customers, key=lambda x: (x[1], x[0])))`
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:I now know the final answer
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion

"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

In [36]:
##############################################################
# View detailed outputs of the chains
###############################################################

In [37]:
import langchain

langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

langchain.debug=False



[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

In [38]:
##############################################################
# Define your own tool
# - Guidance on crafting a custom tool using a "time" 
#   function as an example.
# - Introduction of the "tool decorator" for defining 
#   a function as a tool for LangChain.
# - Emphasis on the importance of clear docstrings 
#   in guiding the agent's use of the tool.
###############################################################

In [39]:
!pip install DateTime


  Obtaining dependency information for DateTime from https://files.pythonhosted.org/packages/ff/d5/f508192a563ab7415d1efbbe8d39cb9f0e510a1f6aaee3ca7d4ffed2a194/DateTime-5.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.1 MB/s eta 0:00:00


In [40]:
from langchain.agents import tool
from datetime import date

In [41]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())


In [42]:
##############################################################
# Integrating Custom Tool with an Agent
# - Instructions on integrating a custom tool 
#   with an agent.
# - Process involves adding the custom tool to 
#   the list of existing tools used by the agent.
# - Example includes using a tool that returns 
#   the current date.
# - Explanation of how the agent recognizes the 
#   need to use the custom tool and interacts with 
#   it to provide the requested information.
##############################################################

In [43]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)


In [44]:
# **Note**: 
# 
# The agent will sometimes come to the wrong 
# conclusion (agents are a work in progress!). 
# 
# If it does, please try running it again.

In [45]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")




> Entering new AgentExecutor chain...
Question: whats the date today?
Thought: I can use the `time` tool to get today's date
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-03-05
Thought:I need to return the date as the final answer.
Final Answer: Today's date is 2024-03-05.

> Finished chain.
